<a href="https://colab.research.google.com/github/jimin0/Graduation_Project_HE_Study/blob/main/ex5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 졸업작품
>SEAL-Python 정리

[TODO]


1.   암호문 type 확인
2.   암호문 + 암호문
3.   암호문 + 비암호문
4.   암호문 * 암호문
5.   암호문 * 비암호문
6.   복호화
7.   재선형화?


## 환경설정

In [4]:
!git clone https://github.com/Huelse/SEAL-Python.git
%cd SEAL-Python

# Install dependencies
!pip3 install numpy pybind11

# Init the SEAL and pybind11
!git submodule update --init --recursive
# Get the newest repositories (dev only)
# git submodule update --remote

# Build the SEAL lib
%cd SEAL
!cmake -S . -B build -DSEAL_USE_MSGSL=OFF -DSEAL_USE_ZLIB=OFF
!cmake --build build
%cd ..

# Run the setup.py
!python3 setup.py build_ext -i


Cloning into 'SEAL-Python'...
remote: Enumerating objects: 1628, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 1628 (delta 175), reused 191 (delta 162), pack-reused 1376
Receiving objects: 100% (1628/1628), 8.66 MiB | 13.25 MiB/s, done.
Resolving deltas: 100% (879/879), done.
/content/SEAL-Python/SEAL-Python
Submodule 'SEAL' (https://github.com/microsoft/SEAL.git) registered for path 'SEAL'
Submodule 'pybind11' (https://github.com/pybind/pybind11.git) registered for path 'pybind11'
Cloning into '/content/SEAL-Python/SEAL-Python/SEAL'...
Cloning into '/content/SEAL-Python/SEAL-Python/pybind11'...
Submodule path 'SEAL': checked out '206648d0e4634e5c61dcf9370676630268290b59'
Submodule path 'pybind11': checked out 'be97c5a98b4b252c524566f508b5c79410d118c6'
/content/SEAL-Python/SEAL-Python/SEAL
-- Build type (CMAKE_BUILD_TYPE): Release
-- The CXX compiler identification is GNU 11.4.0
-- The C compiler identification is G

## 동형암호

### bgv 객체 생성 및 키 생성

In [2]:
# SEAL 라이브러리의 모든 클래스와 함수를 임포트합니다.
from seal import *
import numpy as np

# BGV 암호화 스킴을 사용할 암호화 매개변수 객체를 생성합니다.
parms = EncryptionParameters(scheme_type.bgv)

"""
poly_modulus_degree가 클수록 암호화된 텍스트의 크기가 커지고 모든 연산이 느려지지만,
더 복잡한 암호화된 계산을 가능하게 함. 권장되는 값들은 1024, 2048, 4096, 8192, 16384, 32768이지만,
이 범위를 넘어가는 것도 가능
"""
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))
parms.set_plain_modulus(8192)

context = SEALContext(parms)

# 키 생성

keygen = KeyGenerator(context)

secret_key = keygen.secret_key()
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()
galois_keys =  keygen.create_galois_keys()

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

### [TODO]

#### 1.암호문 type 확인

In [3]:
# 암호화 및 복호화
plaintext = Plaintext("1000")
print("평문화된 텍스트 값 (문자열):", plaintext.to_string())
print("평문화된 텍스트 객체 주소:", plaintext)
print("평문화된 텍스트 값의 타입:", type(plaintext.to_string()))
print("평문화된 텍스트 객체의 타입:", type(plaintext))

print()
ciphertext = Ciphertext()
ciphertext = encryptor.encrypt(plaintext)
#print("암호화된 텍스트 값 (문자열):", ciphertext.to_string())
print("암호화된 텍스트 객체 주소:", ciphertext)
print("암호화된 텍스트 값의 타입:", type(ciphertext.to_string()))
print("암호화된 객체의 타입:", type(ciphertext))

print()
decrypted = Plaintext()
decryptor.decrypt(ciphertext, decrypted)
print("복호화된 텍스트 값 (문자열):", decrypted.to_string())
print("복호화된 텍스트 객체 주소:", decrypted)
print("복호화된 텍스트 값의 타입:", type(decrypted.to_string()))
print("복호화된 객체의 타입:", type(decrypted))


평문화된 텍스트 값 (문자열): 1000
평문화된 텍스트 객체 주소: <seal.Plaintext object at 0x7f4255c255f0>
평문화된 텍스트 값의 타입: <class 'str'>
평문화된 텍스트 객체의 타입: <class 'seal.Plaintext'>

암호화된 텍스트 객체 주소: <seal.Ciphertext object at 0x7f4255c25270>
암호화된 텍스트 값의 타입: <class 'bytes'>
암호화된 객체의 타입: <class 'seal.Ciphertext'>

복호화된 텍스트 값 (문자열): 1000
복호화된 텍스트 객체 주소: <seal.Plaintext object at 0x7f4255c245f0>
복호화된 텍스트 값의 타입: <class 'str'>
복호화된 객체의 타입: <class 'seal.Plaintext'>


#### 2.암호문 + 암호문
>모듈 2가지
1.   evaluator.add(ciphertext1,ciphertext2) : return 값 있음.
2.   evaluator.add_inaplace(ciphertext1,ciphertext2) : ciphertext+=ciphertext2 (no return)


##### 전체 예제 코드

In [5]:
# 암호화
plaintext1 = Plaintext("6")
plaintext2 = Plaintext("3")

ciphertext1 = encryptor.encrypt(plaintext1)       # 6를 암호화하여 ciphertext1에 저장
ciphertext2 = encryptor.encrypt(plaintext2)       # 3을 암호화하여 ciphertext2에 저장

# 암호문 + 암호문 덧셈 1 (add 모듈)
result_ciphertext = evaluator.add(ciphertext1, ciphertext2)

# 암호문 + 암호문 덧셈 2 (add_inplace 모듈)
evaluator.add_inplace(ciphertext1, ciphertext2)

# 결과 복호화
# result_ciphertext를 복호화하여 그 결과를 decrypted에 저장
decrypted = Plaintext()
decryptor.decrypt(result_ciphertext, decrypted)

print("결과:", decrypted)     # to_string 안할시, 복호화된 배열 주소가 출력됨.
decrypted_result =  decrypted.to_string()
print("결과:",type(decrypted_result))
print("결과:",decrypted_result)

# 문자열 형식을 기반으로 진수 판단
# SEAL은 일반적으로 16진수 형태의 문자열을 반환합니다.
if all(c in '0123456789ABCDEF' for c in decrypted_result.upper()):
    print("복호화된 결과는 16진수를 나타냅니다.")
else:
    print("복호화된 결과의 진수를 판단할 수 없습니다.")


결과: <seal.Plaintext object at 0x7f4255c4d7b0>
결과: <class 'str'>
결과: 9
복호화된 결과는 16진수를 나타냅니다.


##### 코드 요약

In [6]:
# 암호화 초기 단계 : plalintext 객체 생성
plaintext1 = Plaintext("6")
plaintext2 = Plaintext("3")


# Chiphertext
ciphertext1 = encryptor.encrypt(plaintext1)
ciphertext2 = encryptor.encrypt(plaintext2)


# 암호문 + 비암호문 덧셈 (add module 사용)
result_ciphertext = evaluator.add(ciphertext1, ciphertext2)

# 결과 복호화
decrypted = Plaintext()
decryptor.decrypt(result_ciphertext, decrypted) # 인자 type : (ciphertext, plaintext)
print("결과:", decrypted)                        # to_string 안할시, 배열 주소가 출력됨.
print("결과:", decrypted.to_string())

noise_budget_after = decryptor.invariant_noise_budget(result_ciphertext)
print("덧셈 후 노이즈 예산:", noise_budget_after, "bits")

결과: <seal.Plaintext object at 0x7f4256b71db0>
결과: 9
덧셈 후 노이즈 예산: 152 bits


#### 3.암호문 + 비암호문
>모듈 1가지
1.   evaluator.add_plain(ciphertext1,ciphertext2) : return 값 있음.

example

`result_ciphertext = evaluator.add_plain(ciphertext1, plaintext2)`

In [7]:
# 암호화 초기 단계 : plalintext 객체 생성
plaintext1 = Plaintext("6")
plaintext2 = Plaintext("3")


# Chiphertext
ciphertext1 = encryptor.encrypt(plaintext1)

# 암호문 + 비암호문 덧셈 (add_plain 모듈 사용)
result_ciphertext = evaluator.add_plain(ciphertext1, plaintext2)

# 결과 복호화
decrypted = Plaintext()
decryptor.decrypt(result_ciphertext, decrypted) # 인자 type : (ciphertext, plaintext)
print("결과:", decrypted)                         # to_string 안할시, 배열 주소가 출력됨.
print("결과:", decrypted.to_string())

noise_budget_after = decryptor.invariant_noise_budget(result_ciphertext)
print("덧셈 후 노이즈 예산:", noise_budget_after, "bits")

결과: <seal.Plaintext object at 0x7f4255c4f870>
결과: 9
덧셈 후 노이즈 예산: 153 bits


####4.암호문 * 암호문
>모듈 2가지
1.   evaluator.multiply(ciphertext1,ciphertext2) : return 값 있음.
2.   evaluator.multiply_inaplace(ciphertext1,ciphertext2) : ciphertext+=ciphertext2 (no return)

In [8]:
# plaintext 객체 생성
plaintext1 = Plaintext("6")
plaintext2 = Plaintext("3")

# 암호화
ciphertext1 = encryptor.encrypt(plaintext1)
ciphertext2 = encryptor.encrypt(plaintext2)

# 암호문 곱셈 (두가지 방법중 택 1)
#result = evaluator.multiply(ciphertext1, ciphertext2)
evaluator.multiply_inplace(ciphertext1, ciphertext2)

# 결과 복호화
decrypted = Plaintext()
decryptor.decrypt(ciphertext1, decrypted)
print("multiply 모듈 쓴 결과(16진수):", decrypted.to_string())            # 결과 12 이지만, 16진수 12 = 10진수 18임
print("multiply 모듈 쓴 결과(10진수):", int(decrypted.to_string(),16))    # 6진수라서 to_String한 결과는 16진수 형태라서 10진수로 바꿈.

# 곱셈 후 노이즈 예산 확인
noise_budget_after = decryptor.invariant_noise_budget(ciphertext1)
print("곱셈 후 노이즈 예산:", noise_budget_after, "bits")


multiply 모듈 쓴 결과(16진수): 12
multiply 모듈 쓴 결과(10진수): 18
곱셈 후 노이즈 예산: 124 bits


####5.암호문*비암호문
>모듈 2가지
1.   evaluator.multiply_plain(ciphertext1,plaintext2) : return 값 있음.
2.   evaluator.multiply_plain_inaplace(ciphertext1,plaintext2) : (no return)

In [9]:
# plaintext 객체 생성
plaintext1 = Plaintext("6")
plaintext2 = Plaintext("3")

# 암호화
ciphertext1 = encryptor.encrypt(plaintext1)

# 암호문 곱셈 (두가지 방법중 택 1)
#result = evaluator.multiply(ciphertext1, ciphertext2)
evaluator.multiply_plain_inplace(ciphertext1, plaintext2)

# 결과 복호화
decrypted = Plaintext()
decryptor.decrypt(ciphertext1, decrypted)
print("multiply 모듈 쓴 결과(16진수):", decrypted.to_string())            # 결과 12 이지만, 16진수 12 = 10진수 18임
print("multiply 모듈 쓴 결과(10진수):", int(decrypted.to_string(),16))    # 6진수라서 to_String한 결과는 16진수 형태라서 10진수로 바꿈.

# 곱셈 후 노이즈 예산 확인
noise_budget_after = decryptor.invariant_noise_budget(ciphertext1)
print("곱셈 후 노이즈 예산:", noise_budget_after, "bits")


multiply 모듈 쓴 결과(16진수): 12
multiply 모듈 쓴 결과(10진수): 18
곱셈 후 노이즈 예산: 151 bits


####6.재선형화

####7.복호화

decrypted = Plaintext()  -> 복호화 할 변수(plaintext)

decryptor.decrypt(암호문, decrypted)

decrypted.to_string : 복호화한 결과 값 추출

decrypted : 배열주소

In [ ]:
# 결과 복호화
decrypted = Plaintext()
decryptor.decrypt(ciphertext1, decrypted)
print("복호화 결과:", decrypted.to_string())